In [1]:
from pyspark.sql import SparkSession, functions as F, types as T
import os

def create_spark(app_name="5400-news-elt"):
    spark = (
        SparkSession.builder
        .appName(app_name)
        .master("local[*]")
        .config("spark.driver.memory", "12g")
        .config("spark.executor.memory", "12g")
        .config("spark.sql.shuffle.partitions", "400")
        .config("spark.default.parallelism", "400")
        .config("spark.sql.files.maxRecordsPerFile", "200000")
        .config("spark.sql.parquet.compression.codec", "snappy")
        .config("spark.sql.parquet.binaryAsString", "false")
        .config("spark.sql.parquet.int96RebaseModeInWrite", "CORRECTED")
        .config("spark.sql.parquet.datetimeRebaseModeInWrite", "CORRECTED")


        # Snowflake JAR
        .config(
            "spark.jars.packages",
            "net.snowflake:snowflake-jdbc:3.15.1,"
            "net.snowflake:spark-snowflake_2.12:2.11.0-sf-1.5.0"
        )
        .getOrCreate()
    )
    spark.sparkContext.setLogLevel("WARN")
    print("🔥 Spark Ready:", spark.version)
    return spark

# Snowflake config
sfOptions = {
    "sfURL": "pobriux-fyc84817.snowflakecomputing.com",
    "sfUser": "AKAAAAFK",
    "sfAuthenticator": "externalbrowser",
    "sfRole": "ACCOUNTADMIN",
    "sfWarehouse": "SNOWFLAKE_LEARNING_WH",
    "sfDatabase": "SNOWFLAKE_LEARNING_DB",
    "sfSchema": "PUBLIC",
}

def write_sf(df, table_name, mode="overwrite"):
    (
        df.write
        .format("snowflake")
        .options(**sfOptions)
        .option("dbtable", table_name)
        .mode(mode)
        .save()
    )
    print(f"❄ 写入 Snowflake → {table_name} （mode={mode})")

# 路径
BASE = r"D:\Columbia\Fall2025\5400\project\layer"
CSV_PATH = r"D:\Columbia\Fall2025\5400\project\All_external.csv"

BRONZE = os.path.join(BASE, "bronze")
SILVER = os.path.join(BASE, "silver")
GOLD = os.path.join(BASE, "gold")
NLP_SILVER = os.path.join(BASE, "silver_for_nlp")
SENTIMENT_SAMPLE = os.path.join(BASE, "sentiment_sample")

In [2]:
def build_bronze(spark):
    print("📥 Loading CSV...")
    df = (
        spark.read.format("csv")
        .options(
            header="true",
            inferSchema="true",
            multiLine="true",
            escape="\"",
            quote="\"",
            mode="PERMISSIVE"
        )
        .load(CSV_PATH)
    )

    for c, t in df.dtypes:
        if t == "string":
            df = df.withColumn(c, F.trim(F.col(c)))

    if "Date" in df.columns:
        df = df.withColumn("Date", F.to_date("Date", "yyyy-MM-dd"))

    df = df.repartition(400)

    print("💾 Writing Bronze → Snowflake")
    write_sf(df, "NEWS_BRONZE")

    return df

In [3]:
def build_silver(spark):
    df = spark.read.parquet(BRONZE)

    if "Date" in df.columns:
        df = df.filter(F.col("Date").isNotNull())
    if "Stock_symbol" in df.columns:
        df = df.filter(F.col("Stock_symbol").isNotNull())

    df = df.dropDuplicates()

    if "Publisher" in df.columns:
        df = df.withColumn("Publisher_norm", F.upper("Publisher"))
    if "Author" in df.columns:
        df = df.withColumn("Author_norm", F.upper("Author"))

    df = df.withColumn("news_id", F.monotonically_increasing_id())
    df = df.repartition(400, "Stock_symbol", "Date")

    print("💾 Writing Silver → Snowflake")
    write_sf(df, "NEWS_SILVER")

    return df

In [4]:
def build_gold(spark):
    df = spark.read.parquet(SILVER)

    df = df.withColumn("title_len", F.length("Article_title"))

    agg_cols = [
        F.count("*").alias("article_count"),
        F.countDistinct("Publisher").alias("publisher_count"),
        F.avg("title_len").alias("avg_title_len"),
    ]

    for col in ["Textrank_summary", "Lsa_summary", "Luhn_summary", "Lexrank_summary"]:
        if col in df.columns:
            agg_cols.append(F.first(col, ignorenulls=True).alias("sample_summary"))
            break

    gold_df = df.groupBy("Date", "Stock_symbol").agg(*agg_cols)
    gold_df = gold_df.repartition(200, "Stock_symbol")

    print("💾 Writing Gold → Snowflake")
    write_sf(gold_df, "NEWS_GOLD")

    return gold_df

In [5]:
def build_silver_nlp(spark):
    df = spark.read.parquet(SILVER)

    df = df.withColumn(
        "text",
        F.coalesce(
            F.col("Article"),
            F.col("Textrank_summary"),
            F.col("Lsa_summary"),
            F.col("Luhn_summary"),
            F.col("Lexrank_summary"),
            F.col("Article_title"),
        )
    )

    df = df.filter(F.col("text").isNotNull() & (F.length(F.trim("text")) > 0))

    df = df.select(
        "news_id", "Date", "Article_title", "Stock_symbol",
        "Publisher", "Author", "Url", "text"
    ).repartition(200)

    print("🧠 Writing silver_for_nlp → Snowflake")
    write_sf(df, "NEWS_SILVER_FOR_NLP")

    return df

In [6]:
def join_silver_sentiment_sample(spark):
    silver_df = spark.read.parquet(SILVER)
    sent_df = spark.read.parquet(SENTIMENT_SAMPLE)

    silver_with_sent = silver_df.join(sent_df, on="news_id", how="left")
    silver_with_sent = silver_with_sent.repartition(200, "Stock_symbol", "Date")

    print("📤 写入 Snowflake：NEWS_SILVER_WITH_SENT_SAMPLE")
    write_sf(silver_with_sent, "NEWS_SILVER_WITH_SENT_SAMPLE")

    return silver_with_sent

In [7]:
def build_gold_with_sentiment(spark):
    silver_sent = join_silver_sentiment_sample(spark)
    gold_df = spark.read.parquet(GOLD)

    sent_agg = (
        silver_sent
        .groupBy("Date", "Stock_symbol")
        .agg(
            F.avg("sentiment_score_signed").alias("avg_sentiment_score"),
            F.sum(F.when(F.col("sentiment_label") == "POSITIVE", 1).otherwise(0)).alias("pos_article_count"),
            F.sum(F.when(F.col("sentiment_label") == "NEGATIVE", 1).otherwise(0)).alias("neg_article_count"),
            F.count(F.col("sentiment_label")).alias("sentiment_article_count")
        )
    )

    gold_with_sent = (
        gold_df.alias("g")
        .join(sent_agg.alias("s"), on=["Date", "Stock_symbol"], how="left")
    )

    gold_with_sent = gold_with_sent.withColumn(
        "sentiment_coverage",
        F.when(
            F.col("article_count") > 0,
            F.col("sentiment_article_count").cast("double") / F.col("article_count")
        ).otherwise(F.lit(0.0))
    )

    gold_with_sent = gold_with_sent.repartition(200, "Stock_symbol")

    print("🏆 写入 Snowflake：NEWS_GOLD_WITH_SENT")
    write_sf(gold_with_sent, "NEWS_GOLD_WITH_SENT")

    return gold_with_sent

In [8]:
spark = create_spark()

bronze_df = build_bronze(spark)

RuntimeError: Java gateway process exited before sending its port number